In [1]:
import warnings
warnings.filterwarnings("ignore")

from typing import List, Tuple
from helper.helper_functions import load_dataset, save_model, get_features_and_target, encode_all_features
from helper.clfmodel_functions import tune_model, seq_feat_selection, multi_metric_cv, plot_multi_score_cv_results, forward_feat_selection_hypertuning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[inFairness]'
pip install 'aif360[Reductions]'
pip install 'aif360[OptimalTransport]'


### Loading the cleaned dataset

In [2]:
data = load_dataset('../data/assignment2_income_cleaned.xlsx')

### Feature Engineering (encoding) & Train-Test Split

In [3]:
# Splitting the data into features (X) and target (y)
X, y = get_features_and_target(data, 'income')
columns_to_exclude = ['age', 'ability to speak english', 'gave birth this year']
# Encoding the features and target
X_encoded, y_encoded = encode_all_features(X, y, [])
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=42)

In [4]:
X_train.head()

,age,education,workinghours,ability to speak english,workclass_governmental,workclass_no paid work,workclass_private,workclass_self employed,marital status_Divorced,marital status_Husband,...,occupation_Office/Administrative Support,occupation_Production/Assembly,occupation_Protective Services,occupation_Repair/Maintenance,occupation_Sales,"occupation_Science, Engineering, Technology",occupation_Service/Hospitality,occupation_Transport,gave birth this year_No,gave birth this year_Yes
6317,22,16,36,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
740,61,22,40,1,0,0,0,1,1,0,...,0,0,0,0,1,0,0,0,1,0
3781,48,16,40,0,0,0,1,0,1,0,...,1,0,0,0,0,0,0,0,1,0
7850,62,18,65,0,0,0,1,0,0,1,...,0,1,0,0,0,0,0,0,1,0
2963,53,19,44,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0


### Model

Here, we quickly train and evaluate a KNN model with random parameters for demonstration.

In [5]:
# K-Nearest Neighbors model
knn_model = KNeighborsClassifier(n_neighbors=5, metric='minkowski', p=2)
knn_model.fit(X_train, y_train)
knn_preds = knn_model.predict(X_test)
knn_accuracy = accuracy_score(y_test, knn_preds)

print(classification_report(y_test, knn_preds))
print("K-Nearest Neighbors Accuracy:", knn_accuracy)

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1175
           1       0.62      0.62      0.62       625

    accuracy                           0.73      1800
   macro avg       0.71      0.71      0.71      1800
weighted avg       0.74      0.73      0.74      1800

K-Nearest Neighbors Accuracy: 0.735


### Feature Importance using the model itself

In [6]:
if False:
    seq_feat_selection(KNeighborsClassifier(), X_train, y_train)

### Hyperparameter tuning

Here, we define the base parameter grid for our hyperparameter tuning function.

In [7]:
param_grid = {
        'n_neighbors': np.arange(2, 31, 1),
        'weights': ['uniform', 'distance'],
        'p': [1, 2]
    }

Below, we tune the hyperparameters of the KNN model using the defined parameter grid and using all features.

In [8]:
if True:
    best_params, best_model, best_accuracy = tune_model(KNeighborsClassifier(), X_train, y_train, X_test, y_test, param_grid, top_n=10)
    
    print("Best Hyperparameters:", best_params)
    print("Best Model:", best_model)
    print("Best Model Accuracy:", best_accuracy)

Fitting 5 folds for each of 116 candidates, totalling 580 fits
Best Hyperparameters: {'n_neighbors': 22, 'p': 1, 'weights': 'uniform'}
Best Model: KNeighborsClassifier(n_neighbors=22, p=1)
Best Model Accuracy: 0.7605555555555555


In [9]:
if True:
    best_params, best_model, best_composite_metric = tune_model(KNeighborsClassifier(), X_train, y_train, X_test, y_test, param_grid, top_n=10, ensure_fairness=True)
    
    print("Best Hyperparameters:", best_params)
    print("Best Model:", best_model)
    print("Best Model Composite Score:", best_composite_metric)

Fitting 5 folds for each of 116 candidates, totalling 580 fits
Best Hyperparameters: {'n_neighbors': 30, 'p': 2, 'weights': 'uniform'}
Best Model: KNeighborsClassifier(n_neighbors=30)
Best Model Composite Score: 0.7912730130961485


Below, we tune the hyperparameters of the KNN model using the defined parameter grid and using a **subset of features**. We exclude the columns 'age', 'ability to speak english', and 'workclass'.

In [10]:
if True:
    # Splitting the data into features (X) and target (y)
    X_, y_ = get_features_and_target(data, 'income')
    columns_to_exclude = ['age', 'ability to speak english', 'workclass']
    X_ = X_.drop(columns=columns_to_exclude)
    # Encoding the features and target, and excluding some columns
    X_encoded_, y_encoded_ = encode_all_features(X_, y_, columns_to_exclude)
    X_train_, X_test_, y_train_, y_test_ = train_test_split(X_encoded_, y_encoded_, test_size=0.2, random_state=42)
    
    best_params, best_model, best_accuracy = tune_model(KNeighborsClassifier(), X_train_, y_train_, X_test_, y_test_, param_grid)
    
    print("\nBest Hyperparameters:", best_params)
    print("Best Model:", best_model)
    print("Best Model Accuracy:", best_accuracy)

Fitting 5 folds for each of 116 candidates, totalling 580 fits

Best Hyperparameters: {'n_neighbors': 27, 'p': 1, 'weights': 'uniform'}
Best Model: KNeighborsClassifier(n_neighbors=27, p=1)
Best Model Accuracy: 0.7738888888888888


In [11]:
# Forward feature selection with hyperparameter tuning
if True:
    best_subset, best_params, best_score = forward_feat_selection_hypertuning(KNeighborsClassifier(), param_grid, X_train, y_train, X_test, y_test)
    
    print("Best subset of features:", best_subset)
    print("Best hyperparameters:", best_params)
    print("Best model accuracy:", best_score)
    
    # Use the best subset and best hyperparameters for final model
    final_model = KNeighborsClassifier(**best_params)
    final_model.fit(X_train[best_subset], y_train)
    final_model_preds = final_model.predict(X_test[best_subset])
    final_model_accuracy = accuracy_score(y_test, final_model_preds)
    
    print(classification_report(y_test, final_model_preds))

Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Fitting 5 folds for each of 116 candidates, totalling 580 fits
Best subset: ['education']
Remaining features: [['age'], ['workinghours'], ['ability to speak english'], ['workclass_governmental', 'workclass_no paid work', 'workclass_private', 'workclass_self employed'], ['marital status_Divorced', 'marital status_Husband', 'marital status_Never married', 'marital status_Separated', 'marital status_Widowed', 'marital status_Wife'], ['occupation_Construction/Extraction', 'occupation_Counseling/

In [12]:
# Define the parameter grid for AdaBoost
param_grid = {
    'algorithm': ['SAMME', 'SAMME.R'],
    'n_estimators': [50, 75, 100, 150],
    'learning_rate': [0.1, 0.5, 1.0],
    'estimator': [final_model, KNeighborsClassifier()],
    'random_state': [42]
}

best_params, best_model, best_accuracy = tune_model(AdaBoostClassifier(), X_train, y_train, X_test, y_test, param_grid)

print("\nBest Hyperparameters:", best_params)
print("Best Model:", best_model)
print("Best Model Accuracy:", best_accuracy)

Fitting 5 folds for each of 48 candidates, totalling 240 fits


ValueError: 
All the 240 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 149, in fit
    self._validate_estimator()
  File "F:\PyCharmProjects\DataMining\.venv\Lib\site-packages\sklearn\ensemble\_weight_boosting.py", line 537, in

In [ ]:
if False:
    # Define the scorers
    scoring = {"AUC": "roc_auc", "Accuracy": make_scorer(accuracy_score)}
    
    n_neighbours_param_min = 2
    n_neighbours_param_max = 31
    param_grid = {"n_neighbors": range(n_neighbours_param_min, n_neighbours_param_max, 1)}

    results = multi_metric_cv(KNeighborsClassifier(p=2), scoring, X_train, y_train, param_grid)
    plot_multi_score_cv_results("n_neighbors", n_neighbours_param_min, n_neighbours_param_max, 0.7, 1, results, "param_n_neighbors", int, scoring)
    
    results = multi_metric_cv(KNeighborsClassifier(p=1), scoring, X_train, y_train, param_grid)
    plot_multi_score_cv_results("n_neighbors", n_neighbours_param_min, n_neighbours_param_max, 0.7, 1, results, "param_n_neighbors", int, scoring)

Feature Selection: Ensure that sensitive attributes such as sex are not included as features in the model. This helps prevent the model from directly learning discriminatory patterns based on sensitive attributes.

Fair Representation: Balance the representation of different groups within the dataset. Ensure that the dataset is representative of the population with respect to sex and other sensitive attributes. This can help mitigate bias in the model's predictions.

Fair Preprocessing: Apply preprocessing techniques that aim to mitigate bias in the dataset. For example, use techniques such as reweighting or resampling to balance the dataset with respect to sensitive attributes.

Reweighing:
Adjust the weights of instances in your dataset to balance the impact of different groups.
Assign higher weights to underrepresented groups (e.g., females) and lower weights to overrepresented groups (e.g., males).
This helps reduce bias during model training.
Suppression:
Remove or suppress the sensitive attribute (‘sex’) from the dataset.
Train your model without using this attribute.
Note that this approach may lead to information loss, so use it cautiously.
Massaging the Dataset:
Modify class labels to achieve fairness.
For instance, you can swap the labels for different groups (e.g., relabel some ‘male’ instances as ‘female’ and vice versa).
This ensures that the model does not learn discriminatory patterns based on the sensitive attribute.
Resampling Techniques:
Oversample the underrepresented group (e.g., females) or undersample the overrepresented group (e.g., males).
This balances the class distribution and reduces bias.


Model Evaluation: Evaluate the model's performance and fairness across different subgroups defined by sensitive attributes such as sex. Use metrics such as disparate impact ratio, equal opportunity difference, or predictive parity to assess fairness.

### Saving the Model

In [ ]:
# Saving the model
save_model(final_model, '../output/saved_models/knn_model.joblib')